In [1]:
devtools::install_github("kasaai/deeptriangle")
devtools::install_github("kasaai/insurance")

ps          (1.7.1     -> 1.7.2    ) [CRAN]
processx    (3.7.0     -> 3.8.0    ) [CRAN]
callr       (3.7.2     -> 3.7.3    ) [CRAN]
matrixStats (0.62.0    -> 0.63.0   ) [CRAN]
vctrs       (0.4.2     -> 0.5.1    ) [CRAN]
RcppEigen   (0.3.3.9.2 -> 0.3.3.9.3) [CRAN]
pkgbuild    (1.2.0     -> 1.4.0    ) [CRAN]
ggplot2     (3.3.6     -> 3.4.0    ) [CRAN]
png         (0.1-7     -> 0.1-8    ) [CRAN]
yaml        (2.3.5     -> 2.3.6    ) [CRAN]
jsonlite    (1.8.2     -> 1.8.3    ) [CRAN]
ggfan       (NA        -> 0.1.3    ) [CRAN]
keras       (2.6.0     -> 2.9.0    ) [CRAN]


Installing 13 packages: ps, processx, callr, matrixStats, vctrs, RcppEigen, pkgbuild, ggplot2, png, yaml, jsonlite, ggfan, keras

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in i.p(...):
“installation of package ‘vctrs’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘jsonlite’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘ggplot2’ had non-zero exit status”


✔  checking for file ‘/tmp/RtmpZxv1wg/remotesee9c88d2/kasaai-deeptriangle-ab2dd55/DESCRIPTION’
─  preparing ‘deeptriangle’:
✔  checking DESCRIPTION meta-information
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘deeptriangle_0.3.0.tar.gz’
   


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)




vctrs (0.4.2 -> 0.5.1) [CRAN]


Installing 1 packages: vctrs

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in i.p(...):
“installation of package ‘vctrs’ had non-zero exit status”


✔  checking for file ‘/tmp/RtmpZxv1wg/remotese46def952/kasaai-insurance-f323fa1/DESCRIPTION’
─  preparing ‘insurance’:
✔  checking DESCRIPTION meta-information
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘insurance_0.2.0.tar.gz’
   


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [2]:
install.packages("GA")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [3]:
library(recipes)
library(insurance)
library(tidyverse)
library(deeptriangle)
library(keras)
library(tensorflow)
library(GA)
library(tictoc)

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘recipes’


The following object is masked from ‘package:stats’:

    step


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6     ✔ purrr   0.3.5
✔ tibble  3.1.8     ✔ stringr 1.4.1
✔ tidyr   1.2.1     ✔ forcats 0.5.2
✔ readr   2.1.3     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()  masks stats::filter()
✖ stringr::fixed() masks recipes::fixed()
✖ dplyr::lag()     masks stats::lag()
Loading required package: foreach


Attaching package: ‘foreach’


The following objects are masked from ‘package:purrr’:

    accumulate, when


Loading required package: iterators

Package 'GA' version 3.2.3
Type 'citation("GA")' for citing this R package

In [4]:
set.seed(1234)

In [5]:
#' Given a time series, return a list
#'  where each element is a vector representing a window
#'  of the time series determined by the offsets
make_series <- function(v, start_offset, end_offset, na_pad = -99) {
  prepad_mask <- function(v, l = 9) {
    length_diff <- l - length(v)
    if (length_diff > 0) {
      c(rep(na_pad, length_diff), v)
    } else {
      v
    }
  }
  
  purrr::map(
    seq_along(v),
    function(x) {
      start <- max(0, x + start_offset)
      end <- max(0, x + end_offset)
      out <- v[start:end]
      ifelse(is.na(out), na_pad, out)
    } %>%
      prepad_mask()
  )
}

mutate_series <- function(data, timesteps = 9) {
  data %>%
    dplyr::group_by(.data$lob, .data$group_code, .data$accident_year) %>%
    dplyr::arrange(.data$lob, .data$group_code, .data$accident_year, .data$development_lag) %>%
    mutate(
      paid_lags = make_series(incremental_paid, -timesteps, -1),
      case_lags = make_series(case_reserves, -timesteps, -1),
      paid_target = make_series(incremental_paid, 0, timesteps - 1),
      case_target = make_series(case_reserves, 0, timesteps - 1),
    ) %>%
    ungroup()
}

prep_keras_data <- function(data, company_index_recipe) {
  lags <- data %>%
    select(.data$paid_lags, .data$case_lags) %>%
    purrr::transpose() %>%
    purrr::map(~ array(unlist(.x), dim = c(1, 9, 2))) %>%
    abind::abind(along = 1) %>%
    unname()
  
  target_paid <- data %>%
    pull(.data$paid_target) %>%
    flatten_dbl() %>%
    array_reshape(c(nrow(data), 9, 1))
  
  target_case <- data %>%
    pull(.data$case_target) %>%
    flatten_dbl() %>%
    array_reshape(c(nrow(data), 9, 1))
  
  company_input <- bake(company_index_recipe, data) %>% as.matrix()
  
  list(
    x = list(
      ay_seq_input = lags, company_input = company_input
    ),
    y = list(
      paid_output = target_paid,
      case_reserves_output = target_case
    )
  )
}

masked_mse <- function(mask_value) {
  function(y_true, y_pred) {
    keep_value <- k_cast(k_not_equal(y_true, mask_value), k_floatx())
    sum_squared_error <- k_sum(
      k_square(keep_value * (y_true - y_pred)),
      axis = 2
    )
    sum_squared_error / k_sum(keep_value, axis = 2)
  }
}

transform_preds <- function(preds) {
  rows <- 1:dim(preds[[1]])[[1]]
  list(
    # predicted_cumulative_loss = map(rows, ~ preds[[1]][, , 2][.x, ]),
    predicted_loss = map(rows, ~ preds[[1]][, , 1][.x, ]),
    predicted_os = map(rows, ~ preds[[2]][, , 1][.x, ])
  ) %>%
    as_tibble()
}

extract_keras_data <- function(data, lob) {
  data %>%
    filter(lob == !!lob) %>%
    pull(.data$keras_data) %>%
    flatten()
}

extract_data <- function(data, lob) {
  data %>%
    filter(lob == !!lob) %>%
    unnest(.data$data)
}

compute_predictions <- function(model, test_data, lob) {
  predictions <- model %>%
    predict(extract_keras_data(test_data, lob)$x) %>%
    transform_preds()
  
  test_data %>%
    unnest(data) %>%
    select(group_code, accident_year, development_lag) %>%
    bind_cols(predictions) %>%
    unnest(predicted_loss, predicted_os) %>%
    group_by(group_code, accident_year) %>%
    mutate(development_lag = development_lag + row_number() - 1) %>%
    filter(development_lag <= 10) %>%
    right_join(
      data_with_features %>%
        filter(lob == !!lob),
      by = c("group_code", "accident_year", "development_lag")
    ) %>%
    arrange(group_code, accident_year, development_lag) %>%
    mutate(
      predicted_os = if_else(
        is.na(predicted_os),
        case_reserves, predicted_os
      ),
      predicted_loss = if_else(
        is.na(predicted_loss),
        incremental_paid, predicted_loss
      )
    ) %>%
    mutate(
      predicted_cumulative_loss = cumsum(predicted_loss) * earned_premium_net,
      predicted_os = predicted_os * earned_premium_net,
      case_reserves_actual = case_reserves_actual * earned_premium_net
    ) %>%
    ungroup() %>%
    gather(
      "type", "value", predicted_cumulative_loss, predicted_os,
      cumulative_paid_loss, case_reserves_actual,
      na.rm = TRUE
    ) %>%
    mutate(
      obs_type = case_when(
        grepl("predicted", type) ~ "prediction",
        calendar_year <= 1997 ~ "observed",
        calendar_year > 1997 ~ "holdout",
        TRUE ~ "observed"
      )
    )
}

In [6]:
dt_model <- function() {
  ay_seq_input <- layer_input(shape = list(9, 2), name = "ay_seq_input")
  company_code_input <- layer_input(shape = 1, name = "company_input")
  company_code_embedding <- company_code_input %>%
    layer_embedding(200, 49, name = "company_code_embedding") %>%
    layer_flatten()%>%
    layer_repeat_vector(9)
  
  encoded <- ay_seq_input %>%
    layer_masking(-99) %>%
    layer_gru(units = 128, dropout = 0.2, recurrent_dropout = 0.04108997)
  
  ay_decoded <- encoded %>%
    layer_repeat_vector(9) %>%
    layer_gru(128, return_sequences = TRUE, dropout = 0.2, recurrent_dropout = 0.09526625)
  
  decoded=layer_concatenate(list(ay_decoded, company_code_embedding))
    
  case_reserves_output <- decoded %>%
    time_distributed(layer_dense(units = 64, activation = "relu")) %>%
    time_distributed(layer_dropout(rate = 0.2290747)) %>%
    time_distributed(layer_dense(units = 1, activation = "relu"), name = "case_reserves_output")
  
  paid_output <- decoded %>%
    time_distributed(layer_dense(units = 64, activation = "relu")) %>%
    time_distributed(layer_dropout(rate = 0.4862567)) %>%
    time_distributed(layer_dense(units = 1, activation = "relu"), name = "paid_output")
  
  model <- keras_model(
    inputs = c(ay_seq_input, company_code_input),
    outputs = c(paid_output, case_reserves_output)
  )
  
  model
}

In [7]:
data(schedule_p)

data_with_features <- schedule_p %>%
  right_join(dt_group_codes, by = c("lob", "group_code")) %>%
  mutate(case_reserves = incurred_loss - cumulative_paid_loss) %>%
  group_by(lob, group_code, accident_year) %>%
  arrange(lob, group_code, accident_year, development_lag) %>%
  mutate(
    incremental_paid_actual = incremental_paid_loss,
    incremental_paid = ifelse(
      calendar_year <= 1997,
      incremental_paid_actual, NA_real_
    ),
    cumulative_paid_actual = cumulative_paid_loss,
    cumulative_paid = ifelse(
      calendar_year <= 1997,
      cumulative_paid_actual, NA_real_
    ),
    case_reserves_actual = case_reserves,
    case_reserves = ifelse(
      calendar_year <= 1997,
      case_reserves_actual,
      NA_real_
    )
  ) %>%
  ungroup() %>%
  mutate(
    bucket = case_when(
      calendar_year <= 1995 & development_lag > 1 ~ "train",
      calendar_year > 1995 & calendar_year <= 1997 &
        development_lag > 1 ~ "validation",
      calendar_year > 1997 ~ "test"
    )
  ) %>%
  mutate(
    incremental_paid = incremental_paid / earned_premium_net,
    incremental_paid_actual = incremental_paid_actual / earned_premium_net,
    cumulative_paid = cumulative_paid / earned_premium_net,
    cumulative_paid_actual = cumulative_paid_actual / earned_premium_net,
    case_reserves = case_reserves / earned_premium_net,
    case_reserves_actual = case_reserves_actual / earned_premium_net
  )

# Recipe for indexing company code
company_index_recipe <- recipe(~ group_code, data = data_with_features) %>%
  step_integer(group_code, zero_based = TRUE) %>%
  prep()

data_keras <- bind_rows(
  validation_data = data_with_features %>%
    filter(bucket %in% c("train", "validation") | development_lag == 1) %>%
    mutate_series() %>%
    filter(bucket == "validation") %>%
    group_by(lob) %>%
    nest() %>%
    mutate(keras_data = map(data, ~ prep_keras_data(.x, company_index_recipe))),
  full_training_data = data_with_features %>%
    filter(bucket %in% c("train", "validation") | development_lag == 1) %>%
    mutate_series() %>%
    filter(bucket %in% c("train", "validation")) %>%
    group_by(lob) %>%
    nest() %>%
    mutate(keras_data = map(data, ~ prep_keras_data(.x, company_index_recipe))),
  test_data = data_with_features %>%
    filter(calendar_year <= 1998) %>%
    mutate_series() %>%
    filter(bucket == "test", calendar_year == 1998) %>%
    group_by(lob) %>%
    nest() %>%
    mutate(keras_data = map(data, ~ prep_keras_data(.x, company_index_recipe))),
  .id = "data_type"
)

Warning message:
“Use of .data in tidyselect expressions was deprecated in tidyselect 1.2.0.
ℹ Please use `"paid_lags"` instead of `.data$paid_lags`”
Warning message:
“Use of .data in tidyselect expressions was deprecated in tidyselect 1.2.0.
ℹ Please use `"case_lags"` instead of `.data$case_lags`”


In [8]:
lobs <- c("commercial_auto","other_liability", "private_passenger_auto", "workers_compensation")

results <- map_df(lobs, function(lob) {
  data_lob <- data_keras %>%
    filter(lob == !!lob)
  full_training_data_keras <- data_lob %>%
    filter(data_type == "full_training_data") %>%
    pull(keras_data) %>%
    flatten()
  validation_data_keras <- data_lob %>%
    filter(data_type == "validation_data") %>%
    pull(keras_data) %>%
    flatten()
  test_data <- data_lob %>%
    filter(data_type == "test_data")
  
  map_df(1:10, function(run_id) {
    if (run_id %% 10 == 1) k_clear_session()
    cat(sprintf("Training LOB %s run %s: ", lob, run_id))
    start_time <- Sys.time()
    model <- dt_model()
    model %>%
      compile(
        optimizer = optimizer_adam(learning_rate = 0.003549904, amsgrad = TRUE),
        loss = list(masked_mse(-99), masked_mse(-99)),
        loss_weights = c(0.5, 0.5)
      )
    
    cb <- callback_early_stopping(min_delta = 0.001, patience = 30, mode = "min", restore_best_weights = TRUE)
    
    history <- model %>%
      fit(
        x = full_training_data_keras$x,
        y = full_training_data_keras$y,
        validation_data = unname(validation_data_keras),
        batch_size = 2250,
        epochs = 150,
        callbacks = list(cb),
        verbose = 0
      )
    
    training_time <- as.integer(Sys.time() - start_time)
    
    cat(sprintf("training for %d epochs took %d seconds at %s", cb$stopped_epoch, training_time, Sys.time()), "\n")
    
    predictions_table <- compute_predictions(model, test_data, lob) %>%
      mutate(lob = !!lob, run_id = !!run_id)
    
    tibble(
      lob = lob,
      run_id = run_id,
      training_time = training_time,
      predictions_table = list(predictions_table),
      trained_epochs = cb$stopped_epoch
    )
  })
})

Loaded Tensorflow version 2.6.5



Training LOB commercial_auto run 1: training for 41 epochs took 11 seconds at 2022-11-29 18:40:00 


Adding missing grouping variables: `lob`
Warning message:
“unnest() has a new interface. See ?unnest for details.
Try `df %>% unnest(c(predicted_loss, predicted_os))`, with `mutate()` if needed”


Training LOB commercial_auto run 2: training for 59 epochs took 8 seconds at 2022-11-29 18:40:14 


Adding missing grouping variables: `lob`
Warning message:
“unnest() has a new interface. See ?unnest for details.
Try `df %>% unnest(c(predicted_loss, predicted_os))`, with `mutate()` if needed”


...

Adding missing grouping variables: `lob`
Warning message:
“unnest() has a new interface. See ?unnest for details.
Try `df %>% unnest(c(predicted_loss, predicted_os))`, with `mutate()` if needed”


Training LOB workers_compensation run 9: training for 76 epochs took 9 seconds at 2022-11-29 18:46:14 


Adding missing grouping variables: `lob`
Warning message:
“unnest() has a new interface. See ?unnest for details.
Try `df %>% unnest(c(predicted_loss, predicted_os))`, with `mutate()` if needed”


Training LOB workers_compensation run 10: training for 44 epochs took 7 seconds at 2022-11-29 18:46:23 


Adding missing grouping variables: `lob`
Warning message:
“unnest() has a new interface. See ?unnest for details.
Try `df %>% unnest(c(predicted_loss, predicted_os))`, with `mutate()` if needed”


In [9]:
predictions_table <- results  %>%
  pull(predictions_table) %>%
  bind_rows()

In [10]:
#' Compute performance metrics
#'
#' @param predictions Predictions data frame.
#' @export
dt_compute_metrics <- function(predictions) {
  predictions %>%
    dplyr::filter(
      .data$development_lag == 10,
      .data$type %in% c("cumulative_paid_loss", "predicted_cumulative_loss")
    ) %>%
    dplyr::group_by(.data$lob, .data$group_code, .data$type, .data$run_id) %>%
    dplyr::summarize(ultimate = sum(.data$value)) %>%
    dplyr::group_by(.data$lob, .data$group_code, .data$type) %>%
    dplyr::summarize(ultimate = mean(.data$ultimate)) %>%
    tidyr::spread(.data$type, .data$ultimate) %>%
    dplyr::mutate(
      pct_error = (.data$predicted_cumulative_loss - .data$cumulative_paid_loss) /
        .data$cumulative_paid_loss) %>%
    dplyr::ungroup() %>%
    dplyr::group_by(.data$lob) %>%
    dplyr::summarize(
      mape = mean(abs(.data$pct_error)),
      rmspe = sqrt(mean(.data$pct_error ^ 2))
    ) %>%
    dplyr::mutate(model = "DeepTriangle")
}

#' Tabulate performance metrics for all models
#'
#' @param data Model results in tidy format.
#' @param metric Performance metric.
#' @export
dt_tabulate_metrics <- function(data, metric = c("mape", "rmspe")) {
  metric <- rlang::arg_match(metric)
  data %>%
    dplyr::filter(metric == !!metric) %>%
    dplyr::select(-.data$metric) %>%
    tidyr::spread(.data$model,.data$ value) %>%
    dplyr::select(
      .data$lob, .data$DeepTriangle
    )
}

In [11]:
model_results <- dt_compute_metrics(predictions_table) %>%
  gather(metric, value, mape, rmspe)

`summarise()` has grouped output by 'lob', 'group_code', 'type'. You can
override using the `.groups` argument.
`summarise()` has grouped output by 'lob', 'group_code'. You can override using
the `.groups` argument.


In [12]:
model_results

lob,model,metric,value
<chr>,<chr>,<chr>,<dbl>
commercial_auto,DeepTriangle,mape,0.04396358
other_liability,DeepTriangle,mape,0.10034020
private_passenger_auto,DeepTriangle,mape,0.02309181
workers_compensation,DeepTriangle,mape,0.04077074
commercial_auto,DeepTriangle,rmspe,0.05824160
other_liability,DeepTriangle,rmspe,0.14807261
private_passenger_auto,DeepTriangle,rmspe,0.03739412
workers_compensation,DeepTriangle,rmspe,0.06608241
